In [ ]:
!pip install transformers datasets seqeval

In [2]:
import os
import pickle

import requests
import torch
import numpy as np

In [3]:
np.random.seed(123)

# Prepare Data

In [5]:
def download_data(saving_dir, data_type):
    """
    Function to download the dataset using data_type to specify if we want the train, valid or test.
    """

    # hardcoded url to download the pickled dataset
    root_url = "https://dot-layer.github.io/blog-external-assets/train_rnn/{}.p"

    url = root_url.format(data_type)
    r = requests.get(url)
    os.makedirs(saving_dir, exist_ok=True)

    open(os.path.join(saving_dir, f"{data_type}.p"), "wb").write(r.content)


download_data("./data/", "train")
download_data("./data/", "valid")
download_data("./data/", "test")

In [6]:
# load the data
train_data_ = pickle.load(open("./data/train.p", "rb"))  # 728,789 examples
valid_data_ = pickle.load(open("./data/valid.p", "rb"))  # 182,198 examples
test_data_ = pickle.load(open("./data/test.p", "rb"))  # 100,000 examples

In [7]:
#Reduce train, valid, test data
import random

devide_constant = 300

train_data = random.choices(train_data_, k=len(train_data_)//devide_constant)
valid_data = random.choices(valid_data_, k=len(valid_data_)//devide_constant)
test_data = random.choices(test_data_, k=len(test_data_)//devide_constant)
print('nombre d\'exemple Train = ',len(train_data))
print('nombre d\'exemple Valid = ',len(valid_data))
print('nombre d\'exemple Test = ',len(test_data))

nombre d'exemple Train =  2429
nombre d'exemple Valid =  607
nombre d'exemple Test =  333


In [18]:
def split_Xy(L):
  X, y = [], []
  for item in L:
    X.append(item[0].split(' '))
    y.append(item[1])
  return X, y

X_train, y_train = split_Xy(train_data)
X_valid, y_valid = split_Xy(valid_data)
X_test, y_test = split_Xy(test_data)

In [19]:
X_train[0], y_train[0]

(['10', 'brant', 'ave', 'kingston', 'on', 'k7k', '4v1'],
 ['StreetNumber',
  'StreetName',
  'StreetName',
  'Municipality',
  'Province',
  'PostalCode',
  'PostalCode'])

In [20]:
label_list = ['StreetNumber', 'StreetName', 'Orientation', 'Unit', 'Municipality',
              'Province', 'PostalCode', 'GeneralDelivery']

tag2id = {tag: id for id, tag in enumerate(label_list)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [21]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(X_train, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(X_valid, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
test_encodings = tokenizer(X_test, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [ ]:
def encode_tags(encodings, tags):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_encodings, y_train)
val_labels = encode_tags(val_encodings, y_valid)
test_labels = encode_tags(test_encodings, y_test)

In [ ]:
class AddressDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = AddressDataset(train_encodings, train_labels)
val_dataset = AddressDataset(val_encodings, val_labels)
test_dataset = AddressDataset(test_encodings, test_labels)

# Fine Tuning

In [ ]:
from transformers import DistilBertForTokenClassification
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(label_list))

Downloading:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this 

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

batch_size = 64

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "epoch",
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=batch_size,  # batch size per device during training
    per_device_eval_batch_size=batch_size,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2429
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 190
  Number of trainable parameters = 65197064
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.908000,1.727550
2,0.929900,0.497797
3,0.101800,0.053808
4,0.023700,0.025667
5,0.020300,0.023116


***** Running Evaluation *****
  Num examples = 607
  Batch size = 64
***** Running Evaluation *****
  Num examples = 607
  Batch size = 64
***** Running Evaluation *****
  Num examples = 607
  Batch size = 64
***** Running Evaluation *****
  Num examples = 607
  Batch size = 64
***** Running Evaluation *****
  Num examples = 607
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=190, training_loss=0.683786163361449, metrics={'train_runtime': 2009.2628, 'train_samples_per_second': 6.045, 'train_steps_per_second': 0.095, 'total_flos': 105383667093600.0, 'train_loss': 0.683786163361449, 'epoch': 5.0})

# Evaluation

In [ ]:
from datasets import load_metric

metric = load_metric("seqeval")

<ipython-input-21-edef154daec7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

***** Running Prediction *****
  Num examples = 333
  Batch size = 64
The following columns in the test set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: offset_mapping. If offset_mapping are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.


In [ ]:
print("Quelques exemples où le modèle tort sur les données de test: \n")
for i in range(0, len(X_test)):
  if true_labels[i] != true_predictions[i]:
    print("Adresse: ", X_test[i], '\n')
    print(" \t True tag: ", true_labels[i], '\n')
    print(" \t pred tag: ", true_predictions[i], '\n')

Quelques exemples où le modèle tort sur les données de test: 

Adresse:  ['1035', 'moncton', 'avenue', 'apt', 'r', 'kamloops', 'british', 'columbia', 'v2b', '1s7'] 

 	 True tag:  ['StreetNumber', 'StreetName', 'StreetName', 'Unit', 'Unit', 'Municipality', 'Province', 'Province', 'PostalCode', 'PostalCode'] 

 	 pred tag:  ['StreetNumber', 'StreetName', 'StreetName', 'StreetName', 'StreetName', 'Municipality', 'Province', 'Province', 'PostalCode', 'PostalCode'] 

Adresse:  ['254', 'willow', 'street', 'lower', 'level', 'kamloops', 'b', 'c', 'v2b', '4c9'] 

 	 True tag:  ['StreetNumber', 'StreetName', 'StreetName', 'Unit', 'Unit', 'Municipality', 'Province', 'Province', 'PostalCode', 'PostalCode'] 

 	 pred tag:  ['StreetNumber', 'StreetName', 'StreetName', 'StreetName', 'Unit', 'Municipality', 'Province', 'Province', 'PostalCode', 'PostalCode'] 

Adresse:  ['107', 'r', 'de', 'lorimier', '4e', 'étage', 'app', '401', 'gatineau', 'qc', 'j8y', '3e7'] 

 	 True tag:  ['StreetNumber', 'Street

In [ ]:
results = metric.compute(predictions=true_predictions, references=true_labels)

/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: StreetNumber seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: StreetName seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Municipality seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Province seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PostalCode seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.8/dist-packages/seqeval/metric

In [ ]:
print(' Score sur la classe Unit \n \t', results['nit'])
print('\n Score sur la classe StreetNumber \n \t', results['treetNumber'])
print('\n Score sur la classe PostalCode \n \t', results['ostalCode'])
print('\n Score sur la classe Province \n \t', results['rovince'])
print('\n Score sur la classe StreetName \n \t', results['treetName'])
print('\n Score sur la classe Municipality \n \t', results['unicipality'])
print('\n ********************* ')
print('\n overall PRECISION sur les données de test \t', results['overall_precision'])
print('\n overall RECALL sur les données de test \t', results['overall_recall'])
print('\n overall ACCURACY sur les données de test \t', results['overall_accuracy'])

 Score sur la classe Unit 
 	 {'precision': 0.9310344827586207, 'recall': 0.9, 'f1': 0.9152542372881356, 'number': 30}

 Score sur la classe StreetNumber 
 	 {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 312}

 Score sur la classe PostalCode 
 	 {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 333}

 Score sur la classe Province 
 	 {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 333}

 Score sur la classe StreetName 
 	 {'precision': 0.9917808219178083, 'recall': 0.9986206896551724, 'f1': 0.995189003436426, 'number': 725}

 Score sur la classe Municipality 
 	 {'precision': 0.990990990990991, 'recall': 0.990990990990991, 'f1': 0.990990990990991, 'number': 333}

 ********************* 

 overall PRECISION sur les données de test 	 0.9946859903381643

 overall RECALL sur les données de test 	 0.9966118102613747

 overall ACCURACY sur les données de test 	 0.9971590909090909


# Bi-LSTM baseline

In [ ]:
# You need the dev version since we need a not-release feature as of January 28, 2022.
# Will remove comments when the feature is released.
%pip install --upgrade git+https://github.com/GRAAL-Research/poutyne.git@dev #install poutyne
%pip install --upgrade colorama #install colorama
%pip install --upgrade pymagnitude-light #install pymagnitude-light
%matplotlib inline

import gzip
import os
import pickle
import shutil
import warnings

import requests
import torch
import torch.nn as nn
import torch.optim as optim
from poutyne import set_seeds
from poutyne.framework import Experiment
from pymagnitudelight import Magnitude
from torch.nn.functional import cross_entropy
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence, pad_sequence
from torch.utils.data import DataLoader

In [9]:
dimension = 300
num_layer = 1
bidirectional = False

lstm_network = nn.LSTM(
    input_size=dimension,
    hidden_size=dimension,
    num_layers=num_layer,
    bidirectional=bidirectional,
    batch_first=True,
)

In [10]:
input_dim = dimension  # the output of the LSTM
tag_dimension = 8

fully_connected_network = nn.Linear(input_dim, tag_dimension)

In [11]:
def download_from_url(model: str, saving_dir: str, extension: str):
    """
    Simple function to download the content of a file from a distant repository.
    """
    print("Downloading the model.")
    model_url = "https://graal.ift.ulaval.ca/public/deepparse/{}." + extension
    url = model_url.format(model)
    r = requests.get(url)

    os.makedirs(saving_dir, exist_ok=True)
    open(os.path.join(saving_dir, f"{model}.{extension}"), "wb").write(r.content)


def download_fasttext_magnitude_embeddings(saving_dir):
    """
    Function to download the magnitude pre-trained fastText model.
    """
    model = "fasttext"
    extension = "magnitude"
    file_name = os.path.join(saving_dir, f"{model}.{extension}")
    if not os.path.isfile(file_name):
        warnings.warn(
            "The fastText pre-trained word embeddings will be download in magnitude format (2.3 GO), "
            "this process will take several minutes."
        )
        extension = extension + ".gz"
        download_from_url(model=model, saving_dir=saving_dir, extension=extension)
        gz_file_name = file_name + ".gz"
        print("Unzip the model.")
        with gzip.open(os.path.join(saving_dir, gz_file_name), "rb") as f:
            with open(os.path.join(saving_dir, file_name), "wb") as f_out:
                shutil.copyfileobj(f, f_out)
        os.remove(os.path.join(saving_dir, gz_file_name))
    return file_name


class EmbeddingVectorizer:
    def __init__(self, path="./"):
        """
        Embedding vectorizer
        """
        file_name = download_fasttext_magnitude_embeddings(saving_dir=path)
        self.embedding_model = Magnitude(file_name)

    def __call__(self, address):
        """
        Convert address to embedding vectors
        :param address: The address to convert
        :return: The embeddings vectors
        """
        embeddings = []
        for word in address.split():
            embeddings.append(self.embedding_model.query(word))
        return embeddings


embedding_vectorizer = EmbeddingVectorizer()

<ipython-input-11-035f21d92b93>:22: UserWarning: The fastText pre-trained word embeddings will be download in magnitude format (2.3 GO), this process will take several minutes.
  warnings.warn(


Unzip the model.


In [12]:
class DatasetBucket:
    def __init__(self, data, embedding_vectorizer):
        self.data = data
        self.embedding_vectorizer = embedding_vectorizer
        self.tags_set = {
            "StreetNumber": 0,
            "StreetName": 1,
            "Unit": 2,
            "Municipality": 3,
            "Province": 4,
            "PostalCode": 5,
            "Orientation": 6,
            "GeneralDelivery": 7,
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):  # We vectorize when data is asked
        data = self.data[item]
        return self._item_vectorizing(data)

    def _item_vectorizing(self, item):
        address = item[0]
        address_vector = self.embedding_vectorizer(address)

        tags = item[1]
        idx_tags = self._convert_tags_to_idx(tags)

        return address_vector, idx_tags

    def _convert_tags_to_idx(self, tags):
        idx_tags = []
        for tag in tags:
            idx_tags.append(self.tags_set[tag])
        return idx_tags


train_dataset_vectorizer = DatasetBucket(train_data, embedding_vectorizer)
valid_dataset_vectorizer = DatasetBucket(valid_data, embedding_vectorizer)
test_dataset_vectorizer = DatasetBucket(test_data, embedding_vectorizer)

In [13]:
address, tag = train_dataset_vectorizer[0]  # Unpack the first tuple

In [14]:
print(f"Tag is now a list of integers : {tag}")

Tag is now a list of integers : [0, 1, 1, 1, 3, 3, 4, 5]


In [24]:
batch_size = 64 # meme que transformer
lr = 0.1

In [16]:
def pad_collate_fn(batch):
    """
    The collate_fn that can add padding to the sequences so all can have
    the same length as the longest one.

    Args:
        batch (List[List, List]): The batch data, where the first element
        of the tuple is the word idx and the second element are the target
        label.

    Returns:
        A tuple (x, y). The element x is a tensor of packed sequence .
        The element y is a tensor of padded tag indices. The word vectors are
        padded with vectors of 0s and the tag indices are padded with -100s.
        Padding with -100 is done because of the cross-entropy loss and the
        accuracy metric ignores the targets with values -100.
    """

    # This gets us two lists of tensors and a list of integer.
    # Each tensor in the first list is a sequence of word vectors.
    # Each tensor in the second list is a sequence of tag indices.
    # The list of integer consist of the lengths of the sequences in order.
    sequences_vectors, sequences_labels, lengths = zip(
        *[
            (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
            for (seq_vectors, labels) in sorted(
                batch, key=lambda x: len(x[0]), reverse=True
            )
        ]
    )

    lengths = torch.LongTensor(lengths)

    padded_sequences_vectors = pad_sequence(
        sequences_vectors, batch_first=True, padding_value=0
    )
    pack_padded_sequences_vectors = pack_padded_sequence(
        padded_sequences_vectors, lengths.cpu(), batch_first=True
    )  # We pack the padded sequence to improve the computational speed during training

    padded_sequences_labels = pad_sequence(
        sequences_labels, batch_first=True, padding_value=-100
    )

    return pack_padded_sequences_vectors, padded_sequences_labels

In [17]:
train_loader = DataLoader(
    train_dataset_vectorizer,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=pad_collate_fn,
    num_workers=4,
)
valid_loader = DataLoader(
    valid_dataset_vectorizer,
    batch_size=batch_size,
    collate_fn=pad_collate_fn,
    num_workers=4,
)
test_loader = DataLoader(
    test_dataset_vectorizer,
    batch_size=batch_size,
    collate_fn=pad_collate_fn,
    num_workers=2,
)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [18]:
class RecurrentNet(nn.Module):
    def __init__(self, lstm_network, fully_connected_network):
        super().__init__()
        self.hidden_state = None

        self.lstm_network = lstm_network
        self.fully_connected_network = fully_connected_network

    def forward(self, packed_sequences_vectors):
        """
        Defines the computation performed at every call.

        Shapes:
            packed_sequence_vectors: batch_size * longest_sequence_length (padding), 300

        """
        lstm_out, self.hidden_state = self.lstm_network(packed_sequences_vectors)
        lstm_out, _ = pad_packed_sequence(lstm_out, batch_first=True)

        tag_space = self.fully_connected_network(lstm_out)
        return tag_space.transpose(-1, 1)  # We need to transpose since it's a sequence

In [19]:
dimension = 300
num_layer = 2
bidirectional = True

lstm_network = nn.LSTM(
    input_size=dimension,
    hidden_size=dimension,
    num_layers=num_layer,
    bidirectional=bidirectional,
    batch_first=True,
)

input_dim = dimension * 2  # since bidirectional

fully_connected_network = nn.Linear(input_dim, tag_dimension)

full_network_bi_lstm = RecurrentNet(lstm_network, fully_connected_network)

In [20]:
optimizer = optim.SGD(full_network_bi_lstm.parameters(), lr)

In [21]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [29]:
epoch_number = 150

In [30]:
exp_bi_lstm = Experiment(
    "./",
    full_network_bi_lstm,
    optimizer=optimizer,
    loss_function=cross_entropy,
    batch_metrics=["acc"],
)

exp_bi_lstm.train(train_loader, valid_generator=valid_loader, epochs=epoch_number)

Loading weights from ./checkpoint.ckpt and starting at epoch 101.
Loading optimizer state from ./checkpoint.optim and starting at epoch 101.
Loading random states from ./checkpoint.randomstate and starting at epoch 101.


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
<ipython-input-16-cd9c7a62e961>:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))
<ipython-input-16-cd9c7a62e961>:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list t

Epoch: 101/150 Step: 38/38 100.00% |████████████████████|ETA: 0.00s loss: 0.145240 acc: 94.954124

<ipython-input-16-cd9c7a62e961>:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))


Epoch: 101/150 Step:  1/10  10.00% |██                  |ETA: 30.56s val_loss: 0.108798 val_acc: 96.226418

...

<ipython-input-16-cd9c7a62e961>:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))


Epoch: 150/150 Step:  2/10  20.00% |████                |ETA: 14.85s val_loss: 0.062560 val_acc: 98.051949

<ipython-input-16-cd9c7a62e961>:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))


Epoch: 150/150 Train steps: 38 Val steps: 10 48.16s loss: 0.089095 acc: 97.536878 val_loss: 0.091089 val_acc: 97.184498
Restoring data from ./checkpoint_epoch_146.ckpt


[{'epoch': 101,
  'time': 49.723367712000254,
  'loss': 0.15564924373014222,
  'acc': 95.33199644500996,
  'val_loss': 0.13917053581707953,
  'val_acc': 95.10803127131706},
 {'epoch': 102,
  'time': 62.82551736200003,
  'loss': 0.15640381993568694,
  'acc': 95.43310394538386,
  'val_loss': 0.12470813728930334,
  'val_acc': 95.96698830115933},
 {'epoch': 103,
  'time': 55.559117683000295,
  'loss': 0.1481261820447646,
  'acc': 95.68687125811022,
  'val_loss': 0.12563629743000232,
  'val_acc': 95.63448220619064},
 {'epoch': 104,
  'time': 46.731450330999905,
  'loss': 0.151194211851672,
  'acc': 95.48654361377777,
  'val_loss': 0.12696624711720517,
  'val_acc': 95.49295971538915},
 {'epoch': 105,
  'time': 49.63417510799991,
  'loss': 0.14779870263916223,
  'acc': 95.65522832073465,
  'val_loss': 0.11999674156136332,
  'val_acc': 95.99372536975038},
 {'epoch': 106,
  'time': 48.25509695099936,
  'loss': 0.14464190056008436,
  'acc': 95.66588458724354,
  'val_loss': 0.12794133604702093,
 

In [31]:
exp_bi_lstm.test(test_loader)

Found best checkpoint at epoch: 146
lr: 0.1, loss: 0.0936584, acc: 97.5293, val_loss: 0.0691092, val_acc: 98.0389
Loading checkpoint ./checkpoint_epoch_146.ckpt
Running test


<ipython-input-16-cd9c7a62e961>:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))


Step: 1/6  16.67% |███▎                |ETA: 18.09s test_loss: 0.094659 test_acc: 97.228149

<ipython-input-16-cd9c7a62e961>:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  (torch.FloatTensor(seq_vectors), torch.LongTensor(labels), len(seq_vectors))


Test steps: 6 10.93s test_loss: 0.073938 test_acc: 97.952621                                              


{'time': 10.925234112000908,
 'test_loss': 0.07393832997539201,
 'test_acc': 97.95262058933933}